In [1]:
import theano
from theano.sandbox import cuda
theano.sandbox.cuda.use("gpu0")

/home/irashadow/.pyenv/versions/miniconda-3.9.1/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:541: UserWarning: Theano flag device=gpu* (old gpu back-end) only support floatX=float32. You have floatX=float64. Use the new gpu back-end with device=cuda* for that value of floatX.
  warnings.warn(msg)
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 4007)


In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *

Using Theano backend.


In [3]:
batch_size = 64

In [4]:
%pwd
path = '/home/irashadow/python_workspace/deep_learning_workspace/data/DogAndCat/'

In [5]:
from vgg16 import Vgg16
import vgg16; reload(vgg16)
from vgg16 import vgg_ft

In [6]:
vgg = Vgg16()
vgg.model.summary()

/home/irashadow/.pyenv/versions/miniconda-3.9.1/lib/python2.7/site-packages/keras/layers/core.py:577: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [7]:
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size)
vgg.finetune(batches)
vgg.fit(batches, val_batches, nb_epoch=1)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


# Removing dropout

In [6]:
model = vgg_ft(2)

/home/irashadow/.pyenv/versions/miniconda-3.9.1/lib/python2.7/site-packages/keras/layers/core.py:577: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [7]:
model.load_weights(path+'results/ft1.h5')

In [8]:
layers = model.layers

In [9]:
last_conv_idx = [index for index,layer in enumerate(layers)
                       if type(layer) is Convolution2D][-1]

In [10]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [11]:
last_conv_idx

30

In [12]:
layers[last_conv_idx]

In [72]:
conv_layers = layers[:last_conv_idx+3]
conv_model = Sequential(conv_layers)
#Dense layers - also known as fully connected or 'FC' layers
fc_layers = layers[last_conv_idx+3:]

In [42]:
#print conv_layers

In [43]:
print fc_layers

[<keras.layers.core.Dense object at 0x7f02d2057d90>, <keras.layers.core.Dropout object at 0x7f02d20194d0>, <keras.layers.core.Dense object at 0x7f02d1f90690>, <keras.layers.core.Dropout object at 0x7f02d1f97410>, <keras.layers.core.Dense object at 0x7f02ce158290>]


In [44]:
batches = get_batches(path+'train', shuffle=False, batch_size = batch_size)
val_batches = get_batches(path+'valid', shuffle=False, batch_size = batch_size)

val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [45]:
val_features = conv_model.predict_generator(val_batches, val_batches.nb_sample)

In [46]:
trn_features = conv_model.predict_generator(batches, batches.nb_sample)

In [47]:
save_array(path+'models/' + 'train_convlayer_features.bc', trn_features)
save_array(path+'models/' + 'valid_convlayer_features.bc', val_features)

In [38]:
trn_features = load_array(path+'models/' + 'train_convlayer_features.bc')
val_features = load_array(path+'models/' + 'valid_convlayer_features.bc')

In [48]:
trn_features.shape

(23000, 25088)

For our new fully connected model, we'll create it using the exact architecture as the last layers of Vgg 16, so that we can conventiently copy pre-trained weights over from that model. However, we'll set the dropout layer's p values to zero, so as to effectively remove dropout.

In [49]:
# Copy the weights from the pre-trained model.
# NB: since we're removing dropout, we want to half the weights.
def proc_wgts(layer): return [o/2 for o in layer.get_weights()]

In [50]:
#such a finedly tuned model needs to be updated very slowly!
opt = RMSprop(lr=0.00001, rho=0.7)

In [51]:
#there exist bugs, so the code is different to the Lesson vedio on youtube.
def get_fc_model():
    model = Sequential([
          Dense(4096, activation='relu', input_dim=25088),  
          #Dense(4096, activation='relu', input_dim=conv_layers[-1].output_shape[1]),
          Dropout(0.),
          Dense(4096, activation='relu'),  
          Dropout(0.),
          Dense(2, activation='softmax'),   
        ])
    
    for l1, l2 in zip(model.layers, fc_layers): 
        #print l2
        l1.set_weights((proc_wgts(l2)))
    
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model
        

In [52]:
fc_model = get_fc_model()

In [53]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [54]:
fc_model.fit(trn_features, trn_labels, nb_epoch=8, batch_size=batch_size, validation_data=(val_features, val_labels))

Train on 23000 samples, validate on 2000 samples
Epoch 1/8
23000/23000 [==============================] - 30s - loss: 0.0457 - acc: 0.9853 - val_loss: 0.0404 - val_acc: 0.9885
Epoch 2/8
23000/23000 [==============================] - 31s - loss: 0.0155 - acc: 0.9957 - val_loss: 0.0488 - val_acc: 0.9870
Epoch 3/8
23000/23000 [==============================] - 31s - loss: 0.0067 - acc: 0.9988 - val_loss: 0.0737 - val_acc: 0.9870
Epoch 4/8
23000/23000 [==============================] - 30s - loss: 0.0034 - acc: 0.9997 - val_loss: 0.1456 - val_acc: 0.9820
Epoch 5/8
23000/23000 [==============================] - 30s - loss: 0.0035 - acc: 0.9997 - val_loss: 0.1028 - val_acc: 0.9865
Epoch 6/8
23000/23000 [==============================] - 30s - loss: 0.0033 - acc: 0.9997 - val_loss: 0.1024 - val_acc: 0.9855
Epoch 7/8
23000/23000 [==============================] - 30s - loss: 0.0033 - acc: 0.9997 - val_loss: 0.1013 - val_acc: 0.9855
Epoch 8/8
23000/23000 [==============================] - 30s -

In [55]:
fc_model.save_weights(path+'models/' + 'no_dropout.h5')

In general, once we've arrived at a model that is overfitting, there are five steps we should take to reduce it:

1. Add more data
2. Use data augmentation
3. Use architectures that generalize well
4. Add regularization (dropout skill)
5. Reduce architecture complexity.  

We'll talk about some of these steps in this lesson.

# Data Augmentation
Earlier we mentioned that overfitting is a result of our network having learned too much of the specifics of our training set. In other words, we've created a model that relies too much on the specific qualities in our training set, and is unable to generalize and make predictions for similar images.


Data augmentation is a method of addressing this. Simply put, data augmentation just alters each batch of our images. It does this through flipping, slightly changing hues, stretching, shearing, rotation, etc. and it does this in ways that make sense. By that we mean, it wouldn't make sense to vertically flip an image of a dog for generalization purposes, because people rarely if ever take upside down images of dogs. The same can be said for distortions; you don't want to apply so much as to make an image that is far beyond any reasonable image one would take of a cat or dog.


Keras allows you to implement data augmentation fairly easily by creating a data-augmentation batch generator. When constructing this generator, you have many choices in picking distortion parameters. Unfortunately, there is no quick and easy way to identify the optimal parameters for data augmentation: your best bet is to simply experiment.

In general, data augmentation is always a good idea to reduce overfitting.

In [60]:
# dim_ordering='tf' uses tensorflow dimension ordering,
#   which is the same order as matplotlib uses for display.
# Therefore when just using for display purposes, this is more convenient
gen = image.ImageDataGenerator(rotation_range=10, width_shift_range=0.1, 
       height_shift_range=0.1, width_zoom_range=0.2, shear_range=0.15, zoom_range=0.1, 
       channel_shift_range=10., horizontal_flip=True, dim_ordering='tf')

TypeError: __init__() got an unexpected keyword argument 'width_zoom_range'

In [ ]:
# Create a 'batch' of a single image
img = np.expand_dims(ndimage.imread('cat.jpg'),0)
# Request the generator to create batches from this image
aug_iter = gen.flow(img)

# Adding data augmentation
Let's try adding a small amount of data augmentation, and see if we reduce overfitting as a result. The approach will be identical to the method we used to finetune the dense layers in lesson 2, except that we will use a generator with augmentation configured. Here's how we set up the generator, and create batches from it:

In [ ]:
gen = image.ImageDataGenerator(rotation_range=15, width_shift_range=0.1, 
                               height_shift_range=0.1, zoom_range=0.1, horizontal_flip=True)

In [64]:
batches = get_batches(path+'train', gen, batch_size=batch_size)
# NB: We don't want to augment or shuffle the validation set
val_batches = get_batches(path+'valid', shuffle=False, batch_size=batch_size)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [70]:
fc_model = get_fc_model()

When using data augmentation, we can't pre-compute our convolutional layer features, since randomized changes are being made to every input image. That is, even if the training process sees the same image multiple times, each time it will have undergone different data augmentation, so the results of the convolutional layers will be different.


Therefore, in order to allow data to flow through all the conv layers and our new dense layers, we attach our fully connected model to the convolutional model--after ensuring that the convolutional layers are not trainable:

In [73]:
for layer in conv_model.layers: layer.trainable = False
# Look how easy it is to connect two models together!
conv_model.add(fc_model)

In [ ]:
conv_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [76]:
conv_model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=3, 
                     validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

# Batch Normalization
Another good standard approach to reducing overfitting is Batch Normalization.

In general, the inputs to your neural network should always be normalized. Normalization is a process where given a set of data, you subtract from each element the mean value for that data set and divide it by the data set's standard deviation. By doing so, we put the input values onto the same "scale", in that all the values have been normalized into units of "# of standard deviations from the mean".

The reason we would want to put our inputs onto the same scale is because unbalanced inputs with a large range of magnitudes can typically cause instability in neural networks. An extremely large input can often cascade down through the layers. Typically such an imbalance creates gradients that are also wildly imbalanced, and this makes the optimization process difficult to prevent things like explosion. It also creates imbalanced weights. Normalizing inputs avoids this problem.

Often times with images, we don't worry about dividing by the standard deviation, but just subtract the mean.

Occasionally, these instabilities can arise during training. Imagine that at some point during training, we end up with one extremely large weight. This extremely large weight will then produce an extremely large output value for some element of the output vector, and this imbalance will again pass through the neural network and make it unstable.

One idea is to normalize the activation outputs. Unfortunately, this won't prevent SGD from trying to create an imbalanced weight again during the next back-propagation, and trying to solve the problem this way will just cause SGD to continuously try to under this activation layer normalization. Batch normalize extends this idea with two additions:

- after normalizing the activation layer, let's multiply the outputs by an arbitrarily set parameter, and add to that value an additional parameter, therefore setting a new standard deviation and mean
- Make all four of these values (the mean and standard deviation for normalization, and the two new parameters to set arbitrary mean and standard deviations after the normalization) trainable.

This ensures that the weights don't tend to push very high or very low (since the normalization is included in the gradient calculations, so the updates are aware of the normalization). But it also ensures that if a layer does need to change the overall mean or standard deviation in order to match the output scale, it can do so.

By default, you should always include batch normalization, and all modern neural networks do so because:

- Adding batchnorm to a model can result in 10x or more improvements in training speed
- Because normalization greatly reduces the ability of a small number of outlying inputs to over-influence the training, it also tends to reduce overfitting.

In [5]:
model = vgg_ft(2)

/home/irashadow/.pyenv/versions/miniconda-3.9.1/lib/python2.7/site-packages/keras/layers/core.py:577: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [6]:
model.load_weights(path+'results/ft1.h5')

In [7]:
layers = model.layers

In [8]:
last_conv_idx = [index for index,layer in enumerate(layers)
                       if type(layer) is Convolution2D][-1]

In [9]:
last_conv_idx

30

In [10]:
layers[last_conv_idx]

In [11]:
conv_layers = layers[:last_conv_idx+1]
conv_model = Sequential(conv_layers)
#Dense layers - also known as fully connected or 'FC' layers
fc_layers = layers[last_conv_idx+1:]

In [12]:
print fc_layers

[<keras.layers.pooling.MaxPooling2D object at 0x7fbb9a5b1990>, <keras.layers.core.Flatten object at 0x7fbb9a54c410>, <keras.layers.core.Dense object at 0x7fbb9a53de50>, <keras.layers.core.Dropout object at 0x7fbb9a4d0890>, <keras.layers.core.Dense object at 0x7fbb9a439a50>, <keras.layers.core.Dropout object at 0x7fbb9a444f50>, <keras.layers.core.Dense object at 0x7fbb96645710>]


In [13]:
conv_layers[-1].output_shape[1:]

(512, 14, 14)

In [14]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(p),
        BatchNormalization(),
        Dense(4096, activation='relu'),
        Dropout(p),
        BatchNormalization(),
        Dense(1000, activation='softmax')
        ]

In [15]:
p=0.6

In [16]:
bn_model = Sequential(get_bn_layers(0.6))

In [17]:
def proc_wgts(layer, prev_p, new_p):
    scal = (1-prev_p)/(1-new_p)
    return [o*scal for o in layer.get_weights()]

In [18]:
for l in bn_model.layers: 
    if type(l)==Dense: l.set_weights(proc_wgts(l, 0.3, 0.6))

In [19]:
bn_model.pop()
for layer in bn_model.layers: layer.trainable=False

In [20]:
bn_model.add(Dense(2,activation='softmax'))

In [21]:
bn_model.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])

In [23]:
batches = get_batches(path+'train', shuffle=False, batch_size = batch_size)
val_batches = get_batches(path+'valid', shuffle=False, batch_size = batch_size)

val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [24]:
val_features = conv_model.predict_generator(val_batches, val_batches.nb_sample)

In [25]:
trn_features = conv_model.predict_generator(batches, batches.nb_sample)

In [ ]:
save_array(path+'models/' + 'train_convlayer_features2.bc', trn_features)
save_array(path+'models/' + 'valid_convlayer_features2.bc', val_features)

In [ ]:
bn_model.fit(trn_features, trn_labels, nb_epoch=8, validation_data=(val_features, val_labels))

In [ ]:
bn_model.save_weights(model_path+'bn.h5')

In [ ]:
bn_model.load_weights(model_path+'bn.h5')

In [ ]:
bn_layers = get_bn_layers(0.6)
bn_layers.pop()
bn_layers.append(Dense(2,activation='softmax'))